In [ ]:
import sys
!{sys.executable} -m pip install "qiskit[all]" mqt.ddsim

# Grover Example

In [ ]:
from qiskit import *
from numpy import pi
import numpy as np
from qiskit.algorithms import Grover

## Initialization

In [ ]:
a = QuantumRegister(1, "a")
b = QuantumRegister(1, "b")
c = QuantumRegister(1, "c")
d = QuantumRegister(1, "d")
flag = QuantumRegister(1, "flag")
creg_c = ClassicalRegister(4, "classical")
circuit = QuantumCircuit(a, b, c, d, flag, creg_c)
circuit.draw(output="mpl")

## Superposition of all States

In [ ]:
# put all qubits to superposition
circuit.h(a)
circuit.h(b)
circuit.h(c)
circuit.h(d)

# apply "not" gate to flag. This is important for marking the solution state
circuit.x(flag)
circuit.barrier(a, b, c, d, flag)
circuit.draw(output="mpl")

## Grover Iterations: Oracle and Diffusor / Amplitude Amplification

In [ ]:

for i in range(400):
    # Oracle
    # 3 qubits in the picture before (d != c, b != d, a != b)
    circuit.cx(d, c)
    circuit.cx(b, d)
    circuit.cx(a, b)
    # if b c d are 1, apply phase to flag, which flips the amplitude
    circuit.mcp(np.pi, [b, c, d], flag)
    
    # operation is reversible, Uncompute
    # Oracle takes the input gives the input as an output, and only marks the solution.
    # That's why we have uncompute.
    circuit.cx(a, b)
    circuit.cx(b, d)
    circuit.cx(d, c)
    circuit.barrier(a, b, c, d)
    
    # Diffusor / Amplitude Amplification
    # standard structure of hadamard. always the same: hadamard - x - phase gate - x - hadamard
    circuit.h(a)
    circuit.h(b)
    circuit.h(c)
    circuit.h(d)
    circuit.x(d)
    circuit.x(a)
    circuit.x(b)
    circuit.x(c)
    circuit.mcp(np.pi, [a, b, c], d)
    circuit.x(d)
    circuit.x(c)
    circuit.x(b)
    circuit.x(a)
    circuit.h(d)
    circuit.h(c)
    circuit.h(b)
    circuit.h(a)
    circuit.barrier(a, b, c, d)
    
# circuit.draw(output="mpl", fold=-1)

In [ ]:
# print(circuit.qasm())

## Measurements

In [ ]:
# take the qubit a and measure it to classical register 0
circuit.measure(a, creg_c[0])
circuit.measure(b, creg_c[1])
circuit.measure(c, creg_c[2])
circuit.measure(d, creg_c[3])
# circuit.draw(output="mpl", fold=-1)

## Execution on Simulator

In [ ]:
from mqt import ddsim
from qiskit.visualization import plot_histogram

In [ ]:
backend = ddsim.DDSIMProvider().get_backend("qasm_simulator")
job = execute(circuit, backend, shots=1000)
counts = job.result().get_counts(circuit)
plot_histogram(counts, color="midnightblue", title="Histogram", figsize=(6, 4))

# MaxCut with VQAs
Example is adapted from https://qiskit.org/documentation/optimization/tutorials/06_examples_max_cut_and_tsp.html

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import networkx as nx

from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from qiskit.algorithms.minimum_eigen_solvers import QAOA

## Problem Definition

In [ ]:
n = 4  # Number of nodes in graph
G = nx.Graph()
G.add_nodes_from(np.arange(0, n, 1))
elist = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
# tuple is (i,j,weight) where (i,j) is the edge
G.add_weighted_edges_from(elist)

colors = ["r" for node in G.nodes()]
pos = nx.spring_layout(G)


def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=0.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)


draw_graph(G, colors, pos)

## Mapping problem to suitable format for Quantum Computing

### Representation as Adjacency Matrix

In [ ]:
# Computing the weight matrix from the random graph
w = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i, j, default=0)
        if temp != 0:
            w[i, j] = temp["weight"]
print(w)

### Formulating as a QUBO

In [ ]:
max_cut = Maxcut(w)
qp = max_cut.to_quadratic_program()
print(qp.prettyprint())

### Creating a Hamiltonian from QUBO by transforming it to a Ising Formulation first

In [ ]:
qubitOp, offset = qp.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))

## Use VQE algorithm to solve it

In [ ]:
backend = ddsim.DDSIMProvider().get_backend("qasm_simulator")
quantum_instance = QuantumInstance(backend)

### Select ansatz and initialize VQE solver

In [ ]:
# construct VQE
spsa = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
vqe = VQE(ry, optimizer=spsa, quantum_instance=quantum_instance)
vqe.ansatz.decompose().draw(output="mpl", fold=-1)

### Execute VQE and print Result

In [ ]:
result = vqe.compute_minimum_eigenvalue(qubitOp)

x = max_cut.sample_most_likely(result.eigenstate)
print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", qp.objective.evaluate(x))

In [ ]:
qc = vqe.ansatz.bind_parameters(result.optimal_point)
qc.decompose().draw(output="mpl", fold=-1)

## Visualize Result

In [ ]:
# plot results
colors = ["r" if x[i] == 0 else "c" for i in range(n)]
draw_graph(G, colors, pos)
